# Dynamischer Stromtarif vs. Standardtarif

Dynamische Stromtarife wie z.B. [Tibber][] oder [Tado Hourly][] (ehemals awattar) gelten als Teil der Lösung um die Volatitlät erneuerbarere Energien auszugleichen. Anstatt
eines fixen Preises pro Kilowattstunde bezahlt man einen variablen Preis abhängig von den Produktionskosten. Wenn die Sonne scheint und der Wind weht, sind diese besonders niedrig. 
Wenn man sein Verbrauchsverhalten anpasst und große Verbraucher dann einschaltet, wenn der Strom günstig ist, lassen sich damit die Stromkosten senken. Das funktioniert besonders dann 
gut, wenn man Großverbraucher hat (bspw. eine Wärmepumpe oder ein E-Auto), die sich gut automatisch steuern lassen. 

Aber wie sieht es für einen normalen Haushalt aus? Ich wohne in einem Haus mit 4 Wohnungen, die sich einen Stromanschluss
teilen. Der Verbrauch wird hausintern über Subzähler abgerechnet. Diese Zähler kann ich [digital auslesen](https://github.com/grst/energymeter). 
Für 2024 habe ich die stundengenauen Verbrauchsdaten für alle vier Wohnungen. Damit kann ich berechnen ob sich ein
dynamischer Stromtarif bereits jetzt, ohne steuerbare Großverbraucher rechnen würde. Die Wohnungen haben unterschiedliche
Verbrauchsprofile. Zusätzlich verfügt das Haus seit Juni 2024 über eine 24 kWp Photovoltaikanlage mit 22kWh Speicher. 
In den Sommer und Übergangsmontaten bietet uns das einen Autarkiegrad von fast 100%. Man kauft Netzstrom also haupsächlich
im Winter oder Nachts, also dann wenn es auch im Netz keinen PV Strom gibt und der Strom potenziell teurer. Lohnt 
sich ein dynamischer Stromtarif auch in diesem Fall? 


| Wohnung | Beschreibung |
| -- | -- |
|Wohnung 1 | Ganzjährig bewohnt von einer alleinstehenden Person, ca. 25 Jahre alte Haushaltsgeräte. Außerdem laufen aus historischen Gründen diverse gemeinschaftliche Verbraucher (Treppenhaus, Heizung, ...) auf diesen Zähler|
|Wohnung 2 | Ganzjährig bewohnt von zwei Personen, zusätzliche 3kWp Photovolatikanlage mit 6kWh Speicher hinter dem Zähler, teilweise erneuerte Haushalsgeräte |
|Wohnung 3 | Neubauwohnung mit neuen Haushaltsgeräten, bewohnt von einer Familie mit zwei kleinen Kindern |
|Wohnung 4| Nur sporadisch genutzte Ferienwohnung|


## Verglichene Stromtarife

Es gibt eine Unzahl an unterschiedlichen Stromtarifen in Deutschland mit teilweise sehr unterschiedlichen
Preisstrukturen. Hier betrachte ich nur zwei Tarife: 
 * [AÜW Allgäustrom Basis](https://auew.de/privatkunden/strom/allgaeustrom-basis/), unser bisheriger Anbieter
 * [Tado Hourly][]

Die Tarife gestalten sich wie folgt (alle Preise inkl. MwSt)

### AÜW Allgäustrom Basis

| Jahr | Arbeitspreis pro kWh (inkl. Netzentgelt) | Grundpreis pro Monat |
| -- | -- | -- |
| 2024 | 32,19 ct | 15,68 EUR |

### Tado Hourly

| Arbeitspreis pro kWh | Netzentgelt/Abgaben pro kWh | Grundpreis pro Monat |
| -- | -- | -- |
| Epex Spot + 19% MwSt + 1,785ct | 17,33116 ct | 15,77 EUR |


Die Grundpreise sind sehr ähnlich, daher betrachten wir hier nur die Arbeitspreise. 

## Fragestellungen

* Lohnt sich ein dynamischer Stromtarif für einen Standardhaushalt ohne steuerbare Großverbraucher? 
* Wie verändert sich das Ergebnis über die Jahre hinweg, z.B. während des durch den russischen Angriffskrieg auf 
  die Ukraine verursachte Energiekrise 2022? 

[Tado Hourly]: https://energy.tado.com/
[Tibber]: https://tibber.com/de



In [1]:
#| results: hide
import pandas as pd
from datetime import datetime
import requests
import holoviews as hv

hv.extension("bokeh", logo=False)

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
VAT = 1.19  # VAT for dynamic prices
FIXED_FEE_PER_KWH = 0.19116  # EUR / kWh (brutto) (on top of dynamic price)
FEE_PER_KWH = 0.3322  # EUR / kWh (brutto), incl. network fee, for fixed price tariff
YEARS = [2020, 2021, 2022, 2023, 2024]  # consider market prices from these years

hourly_cost = {}
monthly_cost = {}
yearly_cost = {}

In [3]:
# Get hourly prices from awattar API
begin = datetime.fromisoformat("2020-01-01").timestamp()
end = datetime.now().timestamp()
# API endpoint URL
url = f"https://api.awattar.de/v1/marketdata?start={int(begin)*1000}&end={int(end)*1000}"

# Fetch JSON data from the API
response = requests.get(url)
data = response.json()

# Convert JSON data to Pandas DataFrame
hourly_price = pd.DataFrame(data["data"])

hourly_price["start"] = pd.to_datetime(hourly_price["start_timestamp"], unit="ms")
hourly_price["end"] = pd.to_datetime(hourly_price["end_timestamp"], unit="ms")
hourly_price["marketprice"], hourly_price["unit"] = hourly_price["marketprice"] / 1000, "EUR/kWh"
hourly_price["start_day"] = hourly_price["start"].dt.strftime(
    "%m-%d %H:%M:%S"
)  # day without year (for inter-year comparison)
hourly_price["real_price"] = hourly_price["marketprice"] * VAT + FIXED_FEE_PER_KWH

# Load consumption data from table
pulses_interval = pd.read_csv("./data/verbrauch_2024.csv").rename(columns={"meter_name": "Zähler"})
pulses_interval["time"] = pd.to_datetime(pulses_interval["time"])
pulses_interval["start_day"] = pulses_interval["time"].dt.strftime(
    "%m-%d %H:%M:%S"
)  # day without year (for inter-year comparison)


In [4]:
#| include: false
# dynamic cost over the years

for year in YEARS:
    tmp_hourly_price = hourly_price.loc[
        lambda x: (x["start"] >= datetime.fromisoformat(f"{year}-01-01 00:00"))
        & (x["end"] < datetime.fromisoformat(f"{year+1}-01-01 00:00"))
    ]
    tmp_hourly_cost = pulses_interval.merge(
        tmp_hourly_price,
        how="inner",
        validate="many_to_one",
        left_on="start_day",
        right_on="start_day",
    ).assign(cost=lambda x: (x["real_price"] / 1000) * x["Wh"])

    tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(
        lambda x: x.set_index("time")["cost"].resample("MS").sum()
    )
    tmp_yearly_cost = tmp_hourly_cost.loc[:, ["Zähler", "cost"]].groupby(["Zähler"]).sum()

    hourly_cost[f"tado_{year}"] = tmp_hourly_cost
    monthly_cost[f"tado_{year}"] = tmp_monthly_cost
    yearly_cost[f"tado_{year}"] = tmp_yearly_cost

/home/sturm/tmp/ipykernel_127241/2740315461.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(
/home/sturm/tmp/ipykernel_127241/2740315461.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(
/home/sturm/tmp/ipykernel_127241/2740315461.py:17: DeprecationWarning: DataFrameGroupBy.appl

In [5]:
#| include: false
# fixed cost
tmp_hourly_cost = pulses_interval.merge(
    hourly_price, how="inner", validate="many_to_one", left_on="time", right_on="start"
).assign(cost=lambda x: (x["Wh"] / 1000) * FEE_PER_KWH)
tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(
    lambda x: x.set_index("time")["cost"].resample("MS").sum()
)
tmp_yearly_cost = tmp_hourly_cost.loc[:, ["Zähler", "cost"]].groupby(["Zähler"]).sum()

hourly_cost["AÜW_2024"] = tmp_hourly_cost
monthly_cost["AÜW_2024"] = tmp_monthly_cost
yearly_cost["AÜW_2024"] = tmp_yearly_cost

/home/sturm/tmp/ipykernel_127241/3188713762.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(


In [6]:
yearly_cost_df = pd.concat(
    [df.assign(tariff=k.split("_")[0]) for k, df in yearly_cost.items() if "2024" in k]
).sort_values(["Zähler", "tariff"])

In [7]:
yearly_cost_df.pivot_table(values="cost", columns="tariff", index="Zähler").assign(
    **{
        "Ersparnis (EUR)": lambda x: -(x["tado"] - x["AÜW"]),
        "Ersparnis (%)": lambda x: (1 - x["tado"] / x["AÜW"]) * 100,
    }
).style.format('{:.2f}')

tariff,AÜW,tado,Ersparnis (EUR),Ersparnis (%)
Zähler,,,,
Wohnung 1,1138.77,987.58,151.18,13.28
Wohnung 2,687.69,610.25,77.44,11.26
Wohnung 3,567.21,502.83,64.38,11.35
Wohnung 4,172.45,154.61,17.83,10.34


In [8]:
monthly_cost_df = (
    pd.concat([df.assign(tariff=k.split("_")[0]) for k, df in monthly_cost.items() if "2024" in k])
    .sort_values(["Zähler", "tariff"])
    .reset_index()
    .melt(["Zähler", "tariff"], value_name="cost")
    .assign(month=lambda x: pd.to_datetime(x["time"]).dt.month_name(locale="de_DE.utf8"))
    .assign(month=lambda x: pd.Categorical(x["month"], categories=x["month"].unique()))
)

In [9]:
hv.Table(monthly_cost_df, ["tariff", "Zähler", "month"], ["cost"]).to.bars(["month", "tariff"], "cost").opts(
    width=750, xrotation=90, legend_position="right"
)

:HoloMap   [Zähler]
   :Bars   [month,tariff]   (cost)

## Vergleich mit den Vorjahren

stark vereinfacht! 

In [10]:
yearly_cost_df = pd.concat(
    [df.assign(year=k.split("_")[1], tariff=k.split("_")[0]) for k, df in yearly_cost.items()]
).sort_values(["Zähler", "year", "tariff"])

In [11]:
hv.Table(yearly_cost_df, ["tariff", "Zähler", "year"], "cost").to.bars(["year", "tariff"], "cost").opts(
    width=400, xrotation=90, legend_position="right"
)

:HoloMap   [Zähler]
   :Bars   [year,tariff]   (cost)

In [12]:
#| include: false
monthly_cost_df = (
    pd.concat([df.assign(tariff=k) for k, df in monthly_cost.items()])
    .sort_values(["Zähler", "tariff"])
    .reset_index()
    .melt(["Zähler", "tariff"], value_name="cost")
    .assign(month=lambda x: pd.to_datetime(x["time"]).dt.month_name(locale="de_DE.utf8"))
    .assign(month=lambda x: pd.Categorical(x["month"], categories=x["month"].unique()))
)


# Ratio to fixed tariff over the years
def get_ratios(df):
    ratios = []

    for year in YEARS:
        ratio = pd.Series(
            df.loc[lambda x: x["tariff"] == f"tado_{year}", "cost"].values
            / df.loc[lambda x: x["tariff"] == "AÜW_2024", "cost"].values,
            name=str(year),
        )
        ratios.append(ratio)
    return pd.DataFrame(ratios)


monthly_cost_ratio = (
    monthly_cost_df.groupby(["Zähler", "month"])
    .apply(get_ratios)
    .reset_index()
    .rename(columns={"level_2": "year", 0: "ratio"})
)


/home/sturm/tmp/ipykernel_127241/2745426756.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_cost_df.groupby(["Zähler", "month"])
/home/sturm/tmp/ipykernel_127241/2745426756.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_ratios)


In [13]:
#| include: true
hv.HLine(1).opts(color="black", line_width=2, line_dash="dashed") * hv.Table(
    monthly_cost_ratio, ["year", "Zähler", "month"], ["ratio"]
).to.curve("month", "ratio").overlay("year").opts(width=800, legend_position="right")

:HoloMap   [Zähler]
   :Overlay
      .HLine.I     :HLine   [x,y]
      .NdOverlay.I :NdOverlay   [year]
         :Curve   [month]   (ratio)

## Limitierungen

* Es gibt ggf. auch günstigere Fixpreis Stromtarife. Ob man damit günstiger fährt kann sicher jeder selber ausrechnen. 
* Während des Krisenjahres sind wurden teilweise auch die Preise von fixen Stromtarifen stark erhöht. Dafür waren 
  die Netznutzungsgebühren pro kWh noch etwas geringer. Ein exakter Vergleich benötigt genauere Recherchen der tatsächlichen Kosten. 
* Im Laufe des Jahres wurde eine 22 kWp PV Anlage installiert. 
* Die Daten sind *ohne* Optimierung des Stromverbrauchs. Durch entsprechende Verhaltensänderung kann man ggf. mehr sparen. 

## Schlussfolgerungen

 * Dass die Netznutzungsgebühren als Fixpreis pro kWh berechnet werden verwässert das Preissignal. Um den Anreiz zu erhöhen, den Stromverbrauch "smart" zu steuern wäre es sinnvoller die Gebühren abhängig vom Strompreis zu machen.
 * Strompreise sind komplex und für Verbraucher schwer zu durchschauen... Netzentgelte die sich ändern, Strompreise die sich ändern, auch bei Fixpreistarifen, Netzendgelte sind ggf. reduziert bei "steuerbaren" Anlagen wie z.B. Wärmepumpen.  